In [4]:
# Colab용: ver7의 결측(또는 빈칸)만 admissions로 채우기 (subject_id 매칭)
import pandas as pd

# 이미 Drive가 마운트되어 있다면 아래 줄은 생략 가능
# from google.colab import drive
# drive.mount('/content/drive')

# 파일 경로
ver7_path = "/content/drive/MyDrive/cohort_ver7_add_no_ecg.csv"
admissions_path = "/content/drive/MyDrive/admissions.csv"
out_path = "/content/drive/MyDrive/cohort_ver8_fill_admissions.csv"

# 로드
ver7 = pd.read_csv(ver7_path)
admissions = pd.read_csv(admissions_path)

# 동일 컬럼명이 중복된 경우 제거
ver7 = ver7.loc[:, ~ver7.columns.duplicated()]
admissions = admissions.loc[:, ~admissions.columns.duplicated()]

# 필수 키 점검
if "subject_id" not in ver7.columns:
    raise ValueError("ver7에 subject_id 컬럼이 없습니다.")
if "subject_id" not in admissions.columns:
    raise ValueError("admissions에 subject_id 컬럼이 없습니다.")

# 키 타입 정규화(문자열)하여 매칭 안정화
ver7["subject_id"] = ver7["subject_id"].astype(str)
admissions["subject_id"] = admissions["subject_id"].astype(str)

# ver7이 제공한 컬럼 목록(문제에서 알려준 ver7 스키마)
ver7_cols = [
    "subject_id","hadm_id","stay_id","intime","outtime","gender","race",
    "arrival_transport","disposition","first_ecg_time","door_to_ecg_min",
    "admittime","lab_tat_min","door_to_antithrombotic_min","door_to_cath_min",
    "boarding_delay_min","death_flag","los_days","transfer_type","is_emergency_admission"
]
# 실제 파일에 존재하는 것만 사용
ver7_cols = [c for c in ver7_cols if c in ver7.columns]

# admissions에서 ver7과 공통으로 채울 수 있는 컬럼만 선택(subject_id 제외)
fillable_cols = sorted(set(ver7_cols).intersection(admissions.columns) - {"subject_id"})

# admissions 중복 subject_id 처리: admittime이 있으면 최신으로, 없으면 마지막 행
adm = admissions.copy()
if "admittime" in adm.columns:
    # datetime 변환(가능한 경우만)
    adm["_admittime_parsed"] = pd.to_datetime(adm["admittime"], errors="coerce")
    adm = adm.sort_values(["subject_id", "_admittime_parsed"])
    adm = adm.drop_duplicates(subset=["subject_id"], keep="last")
    adm = adm.drop(columns=["_admittime_parsed"])
else:
    adm = adm.drop_duplicates(subset=["subject_id"], keep="last")

# admissions는 subject_id + fillable_cols만 남김
adm = adm[["subject_id"] + fillable_cols]

# 병합
merged = ver7.merge(adm, on="subject_id", how="left", suffixes=("", "_adm"))

# 채움: ver7이 결측(NaN) 또는 빈 문자열("")일 때만 admissions 값으로 채움
def is_blank(s):
    # NaN 또는 길이 0 문자열을 결측으로 간주
    if s.dtype == object:
        return s.isna() | (s.astype(str).str.len() == 0)
    return s.isna()

filled_counts = {}
for col in fillable_cols:
    adm_col = f"{col}_adm"
    if adm_col in merged.columns:
        mask_to_fill = is_blank(merged[col]) & (~merged[adm_col].isna())
        filled_counts[col] = int(mask_to_fill.sum())
        merged.loc[mask_to_fill, col] = merged.loc[mask_to_fill, adm_col]
        merged.drop(columns=[adm_col], inplace=True, errors="ignore")

# 저장
merged.to_csv(out_path, index=False)

print("Saved:", out_path)
print("ver7 rows:", len(ver7))
print("admissions rows:", len(admissions))
print("merged rows:", len(merged))
print("fillable columns:", fillable_cols)
print("filled cell counts per column:", filled_counts)

Saved: /content/drive/MyDrive/cohort_ver8_fill_admissions.csv
ver7 rows: 836
admissions rows: 546028
merged rows: 836
fillable columns: ['admittime', 'hadm_id', 'race']
filled cell counts per column: {'admittime': 371, 'hadm_id': 0, 'race': 0}
